In [3]:
import os
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torchvision import transforms
from torchvision.utils import save_image

In [4]:
#이 코드 파일이 존재하는 working directory에 mlp_img 폴더가 없으면, mlp_img 폴더 생성 
f not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')

In [71]:
#input image 변환을 위한 함수
def to_img_latent(x):
    x = 0.5 * (x + 1)  
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 2, 2)
    return x

In [5]:
#output image 변환을 위한 함수
def to_img(x):
    x = 0.5 * (x + 1)  
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

#### MNIST data 받아오기 

In [43]:
from torch.utils.data import DataLoader  
from torchvision.datasets import MNIST  

In [6]:
num_epochs = 100 #training data 전체(여기서는 6만개)를 딱 한 번 사용했을 때, 한 epoch이 지나갔다고 말함 
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([  #composes several transforms together  
    transforms.ToTensor(), #여기서는 PIL.Image.Image 를 tensor로 바꿔줌 
    transforms.Normalize((0.5,), (0.5,)) #channel 1이므로 각 1개의 element에 해당하는 mean / stdev 
])  


dataset = MNIST('./data', transform=img_transform, download  = True) #default 옵션은 train = True 
#Dataloader은 Iterator을 반환한다
#배치사이즈만큼 데이터를 로드해줌
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True) 

In [7]:
print(dataset)  

Dataset MNIST
    Number of datapoints: 60000
    Split: train
    Root Location: ./data
    Transforms (if any): Compose(
                             ToTensor()
                             Normalize(mean=(0.5,), std=(0.5,))
                         )
    Target Transforms (if any): None


In [68]:
class autoencoder(nn.Module):
    def __init__(self):
        """
        생성자에서 2개의 nn.Sequential 모듈을 생성하고, 멤버변수(self.encoder, self.decoder)로 지정한다.  
        """
        super(autoencoder, self).__init__() # 부모클래스(nn.Module)에서 정의된 메소드를 호출, 생성자 선언 
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(True), #inplace = True 옵션 -> 또 다른 객체를 반환하지 않고 기존 객체를 수정, 메모리를 아주 조금 save 할 수 있다.    
            nn.Linear(128, 64),
            nn.ReLU(True), nn.Linear(64, 12), nn.ReLU(True), nn.Linear(12, 4))
        self.decoder = nn.Sequential(
            nn.Linear(4, 12),
            nn.ReLU(True),  # ReLU를 쓰는 이유는 이게 generation이 아니라 classification이 목적이라서,, 데이터를 많이 버리는 게 좋음 
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), nn.Linear(128, 28 * 28), nn.Tanh()) #마지막 layer에 softmax?

    def forward(self, x):
        """
        순전파 함수에서는 입력 데이터의 Variable을 받아서 출력 데이터의 Variable을
        반환해야 합니다. Variable 상의 임의의 연산자뿐만 아니라 생성자에서 정의한
        모듈을 사용할 수 있습니다.
        """
        x_latent = self.encoder(x)
        x_output = self.decoder(x_latent)
        
        return x_latent, x_output 
    

In [69]:
model = autoencoder()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [74]:
for epoch in range(num_epochs):
    for data in dataloader:
        inputs, label = data #inputs.shape = [128,1,28,28], label = [128]
        inputs = inputs.view(inputs.size(0), -1)  #view는 size변경. inputs.size(0) = 128. 즉 inputs의 size를 [128,784]로 바꿔줌 
        inputs = Variable(inputs) # 
        # ===================forward=====================
        latent, output = model(inputs) #autoencoder내에서 forward 진행됨
        loss = criterion(output, inputs) 
        # ===================backward====================
        optimizer.zero_grad()  
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss.data))
    #
    if epoch % 10 == 0:
        #latent의 이미지 보여주기
        print(latent.shape)
        pic = to_img_latent(latent.cpu().data)  #to_img를 따로 만들어야 할 것 같은데.. 
        save_image(pic, './mlp_img/latent_{}.png'.format(epoch))  
        
        #output의 이미지 보여주기
        print(output.shape)
        pic = to_img(output.cpu().data)  
        save_image(pic, './mlp_img/output_{}.png'.format(epoch))  

torch.save(model.state_dict(), './sim_autoencoder.pth')  

epoch [1/100], loss:0.1397
torch.Size([96, 4])
torch.Size([96, 784])


In [9]:
#print(help(autoencoder))  
#print(dir(autoencoder))
print(nn.Sequential())

Sequential()


In [14]:
#output 가지고 classification 결과 내서, test accuracy 구해야함  